In [ ]:
%pip install -q "openvino>=2023.1.0"

In [ ]:
pip install rdkit-pypi

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit import RDConfig
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
import pandas as pd


In [ ]:
# Replace 'path_to_file.tsv' with the actual file path
file_path = 'logd74.tsv'

# Load the dataset
df = pd.read_csv(file_path, delimiter='\t')

In [ ]:
def smiles_to_fp(smiles, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=n_bits)
    return np.array(fp)

df['fp'] = df['SMILES'].apply(smiles_to_fp)


In [ ]:
X = np.array(list(df['fp']))
y = df['logD7.4'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

model = Net()


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')


In [ ]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

with torch.no_grad():
    predicted = model(X_test_tensor).squeeze()
    mse = mean_squared_error(y_test_tensor, predicted)
    print(f'Test MSE: {mse}')


In [ ]:
torch.save(model.state_dict(), 'lipophilicity_model.pth')


In [ ]:
from rdkit.Chem import Draw

mols = [Chem.MolFromSmiles(smiles) for smiles in df['SMILES'][:5]]
img = Draw.MolsToImage(mols)
img.show()


In [ ]:
import torch
import openvino.runtime as ov
from pathlib import Path

# Load your PyTorch model
model = Net()  # Assuming 'Net' is your model class
model.load_state_dict(torch.load('lipophilicity_model.pth'))
model.eval()



In [ ]:
pip install onnx

In [ ]:
import torch.onnx

dummy_input = torch.randn(1, 2048)  # Adjust the size according to your model input
torch.onnx.export(model, dummy_input, "model.onnx", opset_version=11)


In [ ]:
# Create OpenVINO Core object instance
core = ov.Core()

# Read the ONNX model
ov_model = core.read_model("model.onnx")

# (Optional) Perform any necessary optimizations
compiled_model = core.compile_model(ov_model, "CPU")


In [ ]:
MODEL_DIR = '/content/sample_data'  # Specify your directory as a string
MODEL_NAME = "lipophilicity_openvino"

# Ensure that `ov_model` is the original OpenVINO model object
# Save the OpenVINO model to disk
ov.save_model(ov_model, MODEL_DIR + "/" + f"{MODEL_NAME}.xml")


In [ ]:
import ipywidgets as widgets

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

In [ ]:
# Load OpenVINO model on device
compiled_model = core.compile_model(ov_model, device.value)
compiled_model

In [ ]:
# Assuming you have a function to convert SMILES to fingerprints
def smiles_to_fp(smiles, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=n_bits)
    return np.array(fp)

# Example SMILES string
smiles = "C[C@H](N)C(=O)O"  # Replace with your SMILES string

# Prepare input tensor
fp = smiles_to_fp(smiles)
input_tensor = torch.tensor(fp, dtype=torch.float32).unsqueeze(0)  # Adding batch dimension

# Convert PyTorch tensor to NumPy array
input_numpy = input_tensor.numpy()

# Create OpenVINO tensor from NumPy array
ov_input_tensor = ov.Tensor(input_numpy)

# Run model inference
result = compiled_model([ov_input_tensor])[0]

# Postprocess and display the result
predicted_lipophilicity = result[0]  # Assuming the model outputs a single value
print(f"Predicted Lipophilicity: {predicted_lipophilicity}")


In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
import numpy as np
import torch
import openvino.runtime as ov

# Load your PyTorch model (assuming it's already trained and saved)
model = Net()  # Replace with your model class
model.load_state_dict(torch.load('lipophilicity_model.pth'))
model.eval()

# OpenVINO setup (assuming you have already converted your model)
core = ov.Core()
ov_model = core.read_model('/content/sample_data/lipophilicity_openvino.xml')
compiled_model = core.compile_model(ov_model, "CPU")

def predict_and_visualize(smiles):
    # Convert SMILES to fingerprint
    fp = smiles_to_fp(smiles)
    input_tensor = torch.tensor(fp, dtype=torch.float32).unsqueeze(0)
    input_numpy = input_tensor.numpy()

    # Create OpenVINO tensor
    ov_input_tensor = ov.Tensor(input_numpy)

    # Run model inference
    result = compiled_model([ov_input_tensor])[0]
    predicted_lipophilicity = result[0]

    # Visualize molecule
    mol = Chem.MolFromSmiles(smiles)
    img = Draw.MolToImage(mol)

    return predicted_lipophilicity, img

# Example usage
smiles_list = ["C[C@H](N)C(=O)O", "CCO", "CCN(CC)CC"]  # Replace with your SMILES strings
for smiles in smiles_list:
    lipophilicity, img = predict_and_visualize(smiles)
    print(f"SMILES: {smiles}, Predicted Lipophilicity: {lipophilicity}")
    display(img)
